In [ ]:
import dask.bag as db
from glob import glob
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from nibabel.streamlines import load as load_trk
import nibabel as nib

import dipy.tracking.streamline as dts
import dipy.tracking.utils as dtu
from skimage.transform import resize
import scipy.ndimage.morphology as morph
from scipy.ndimage import gaussian_filter
from scipy.signal import fftconvolve
from numba import jit
import dask.array as da
from dask import delayed
import time
import dipy.data as dpd

In [ ]:
dpd.fetch_bundles_2_subjects()

In [ ]:
b = db.from_sequence([1, 2, 3, 4, 5, 6])
b

In [ ]:
def squarefx(x):
    y = x*x
    return y

In [ ]:
bb = b.map(squarefx)
bb

In [ ]:
bb.compute()

In [ ]:
def loadtxt(txtfile):
    t = open(txtfile).read().splitlines()
    t = list(map(int, t))
    return t

In [ ]:
#try dask on loading text files
b=db.from_sequence(['t1.txt','t2.txt','t3.txt','t4.txt']).map(loadtxt)
b

In [ ]:
b.compute()

In [ ]:
def squarefxlist(x):
    return [i ** 2 for i in x]

In [ ]:
bb=b.map(squarefxlist)
bb.compute()

In [ ]:
def read_sl(fname):
    from nibabel.streamlines import load as load_trk
    import dipy.tracking.utils as dtu
    import numpy as np
    """
    Reads streamlins from file.
    """
    tgram = load_trk(fname)
    sl = list(dtu.move_streamlines(tgram.streamlines,
                                   np.eye(4), tgram.affine))
    return sl

In [ ]:
ls /Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/

In [ ]:
bundle_files = glob('/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/*.trk')

In [ ]:
#do without dask
%time streamlines = read_sl(bundle_files[0]) 
streamlines[0].shape

In [ ]:
bundle_files

In [ ]:
#do with dask
%time streamlines_loaded = db.from_sequence(bundle_files).map(read_sl).compute()

In [ ]:
s1=streamlines_loaded[0]
np.random.shuffle(s1)
s1_selected = s1[0:160]
s1_selected
s1[0][1,:]

In [ ]:
t1_img = nib.load('/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/t1_warped.nii.gz')
vol_shape=t1_img.shape
vol_shape

In [ ]:
def process_sl(streamlines_tract,take_n_sl,vol_shape,size):
    projected_all = np.zeros([size,size,take_n_sl])
    num_neg = 1
    while num_neg > 0:
        np.random.shuffle(streamlines_tract)
        s1_selected = streamlines_tract[:take_n_sl]
        num_neg = np.where(s1_selected<0)
        num_neg = num_neg[0].size
    for sl_idx, sl in enumerate(s1_selected):
        vol = np.zeros(vol_shape, dtype=bool)
        sl = np.round(sl).astype(int).T
        vol[sl[0], sl[1], sl[2]] = 1
        p0 = resize(np.max(vol, 0),(vol_shape[0], vol_shape[0]))
        p1 = resize(np.max(vol, 1),(vol_shape[0], vol_shape[0]))
        p2 = np.max(vol, 2) 
        projected = np.concatenate((p0,p1,p2))
        projected = morph.binary_dilation(projected, iterations=5)
        projected = resize(projected, (size, size)) #expects 3-d, like rgb
        projected_all[:,:,sl_idx]=projected
    return projected_all

In [ ]:
%%time 
streamlines_processed = np.zeros([100,100,160,len(streamline_loaded)])
for i in range(len(streamline_loaded)):
    print(i)
    sl_processed = process_sl(streamlines_loaded[i],160,vol_shape,100)
    streamlines_processed[:,:,:,i] = sl_processed

In [ ]:
%%time
streamlines_processed = db.from_sequence(streamlines_loaded).map(process_sl,160,vol_shape,100).compute()